# Mapping Done

In [5]:
#import spark as sp
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import*
from pyspark.sql.functions import col, coalesce

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

spark = SparkSession.builder.appName("cdw_sapp_branch.json").getOrCreate()
df_branch = spark.read.format("json") \
    .option("header", True) \
    .load("cdw_sapp_branch.json")

In [6]:
spark = SparkSession\
    .builder\
    .appName("capstone")\
    .config("spark.jars","/Users/roy/Downloads/mysql-connector-j-8.0.32/mysql-connector-j-8.0.32.jar")\
    .getOrCreate()

In [7]:
df_branch = spark.read.format("json") \
    .option("header", True) \
    .load("cdw_sapp_branch.json")

df_branch.printSchema()
df_branch.show(2)


root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)

+-----------+-----------+------------+------------+------------+-----------------+----------+--------------------+
|BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------+-----------+------------+------------+------------+-----------------+----------+--------------------+
|  Lakeville|          1|Example Bank|  1234565276|          MN|     Bridle Court|     55044|2018-04-18T16:51:...|
|    Huntley|          2|Example Bank|  1234618993|          IL|Washington Street|     60142|2018-04-18T16:51:...|
+-----------+-----------+------------+------------+------------+-----------

Mapping Logics

In [8]:
#Change the format of phone number to (XXX)XXX-XXXX
df_branch = df_branch.withColumn('BRANCH_PHONE', concat(lit('('), df_branch['BRANCH_PHONE'].substr(1,3), lit(')'),df_branch['BRANCH_PHONE'].substr(4,3), lit('-'),df_branch['BRANCH_PHONE'].substr(7,4)))

df_branch.show(2)

+-----------+-----------+------------+-------------+------------+-----------------+----------+--------------------+
|BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------+-----------+------------+-------------+------------+-----------------+----------+--------------------+
|  Lakeville|          1|Example Bank|(123)456-5276|          MN|     Bridle Court|     55044|2018-04-18T16:51:...|
|    Huntley|          2|Example Bank|(123)461-8993|          IL|Washington Street|     60142|2018-04-18T16:51:...|
+-----------+-----------+------------+-------------+------------+-----------------+----------+--------------------+
only showing top 2 rows



In [9]:
# Replace null values in "branch_zip" column with "99999"
df_branch = df_branch.withColumn("branch_zip", when(df_branch["branch_zip"].isNull(), "99999").otherwise(df_branch["branch_zip"]))

# Show the updated DataFrame
df_branch.show(1)



+-----------+-----------+------------+-------------+------------+-------------+----------+--------------------+
|BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|BRANCH_STREET|branch_zip|        LAST_UPDATED|
+-----------+-----------+------------+-------------+------------+-------------+----------+--------------------+
|  Lakeville|          1|Example Bank|(123)456-5276|          MN| Bridle Court|     55044|2018-04-18T16:51:...|
+-----------+-----------+------------+-------------+------------+-------------+----------+--------------------+
only showing top 1 row



In [10]:
df_branch.write.format("jdbc") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "creditcard_capstone.CDW_SAPP_BRANCH") \
          .option("user", "root") \
            .option("password", "password") \
            .mode("overwrite")\
            .save()


In [11]:
# Show the updated DataFrame
df_branch.show(1)

+-----------+-----------+------------+-------------+------------+-------------+----------+--------------------+
|BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|BRANCH_STREET|branch_zip|        LAST_UPDATED|
+-----------+-----------+------------+-------------+------------+-------------+----------+--------------------+
|  Lakeville|          1|Example Bank|(123)456-5276|          MN| Bridle Court|     55044|2018-04-18T16:51:...|
+-----------+-----------+------------+-------------+------------+-------------+----------+--------------------+
only showing top 1 row



In [16]:
# making csv files
import pandas as pd

# convert PySpark DataFrame to Pandas DataFrame
branch_pd = df_branch.toPandas()

# save Pandas DataFrame to CSV file
branch_pd.to_csv('branch.csv', index=False)

branch_pd.head(2)

,BRANCH_CITY,BRANCH_CODE,BRANCH_NAME,BRANCH_PHONE,BRANCH_STATE,BRANCH_STREET,branch_zip,LAST_UPDATED
0,Lakeville,1,Example Bank,(123)456-5276,MN,Bridle Court,55044,2018-04-18T16:51:47.000-04:00
1,Huntley,2,Example Bank,(123)461-8993,IL,Washington Street,60142,2018-04-18T16:51:47.000-04:00


23/03/01 00:46:00 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1800160 ms exceeds timeout 120000 ms
23/03/01 00:46:00 WARN SparkContext: Killing executors is not supported by current scheduler.
23/03/01 00:46:06 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:87)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:643)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1057)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:238)
	at s

In [ ]:
stop spark